In [ ]:
import pandas as pd
import numpy as np  # <-- Adicionado para localizar os índices das classes
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer
import joblib

# 1. Preparação e Limpeza
train = pd.read_csv('train.csv')
# Concatenando Título e Texto para não perder contexto
train['total_content'] = train['title'].fillna('') + " " + train['text'].fillna('')

X = train['total_content']
y = train['label']

# 2. Divisão Hold-out (80/20) com semente fixa para reprodutibilidade
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Vetorização com Bigramas
# ATENÇÃO: Se o seu dataset for em português, troque 'english' por 'portuguese' ou None.
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=15000, stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

# 4. Treinamento do Modelo (Corrigido com max_iter=1000)
model = LogisticRegression(C=10, class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

# 5. Validação Interna e Métricas
y_pred = model.predict(X_val_tfidf)
print("=== Matriz de Confusão ===")
print(confusion_matrix(y_val, y_pred))
print("\n=== Relatório de Classificação ===")
print(classification_report(y_val, y_pred))

# 6. Interpretabilidade (XAI) com LIME
# Criamos um pipeline porque o LIME precisa do texto cru como entrada, e não o texto vetorizado
pipeline = make_pipeline(vectorizer, model)

# CORREÇÃO: Identificando claramente que 0 = True (Real) e 1 = Fake
explainer = LimeTextExplainer(class_names=['True (0)', 'Fake (1)']) 

# Pegando a primeira ocorrência de cada classe corretamente na base de validação
idx_true = np.where(y_val == 0)[0][0] # 0 é a notícia real (ex: Reuters)
idx_fake = np.where(y_val == 1)[0][0] # 1 é a notícia falsa (ex: Sensacionalista)

# Gerando e salvando a explicação da Notícia Real
exp_true = explainer.explain_instance(X_val.iloc[idx_true], pipeline.predict_proba, num_features=10)
exp_true.save_to_file('lime_true_explanation.html')

# Gerando e salvando a explicação da Notícia Fake
exp_fake = explainer.explain_instance(X_val.iloc[idx_fake], pipeline.predict_proba, num_features=10)
exp_fake.save_to_file('lime_fake_explanation.html')

print("\nRelatórios de Interpretabilidade LIME salvos como 'lime_true_explanation.html' e 'lime_fake_explanation.html'")

# 7. Salvando os Pesos e o Vetorizador (Engenharia de Software)
joblib.dump(model, 'modelo_fake_news.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')
print("Modelo e Vetorizador serializados com sucesso (.pkl)")